In [0]:
import os
os.chdir("./drive/My Drive/179skill")

In [0]:
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline         

In [0]:
def logit(z): return 1./(1.+np.exp(-z))

In [0]:
def generate_players(num_of_players, skill_cap):
    ## returns a list of player's skill
    return [round(random.uniform(0,skill_cap),2) for i in range(num_of_players)]


def generate_games(players, num_of_games, scale=0.3, style='pygm'):

    if style == 'pystan':
        
        player1=[]
        player2=[]
        outcome=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )

            ##pystan player id is ONE based
            player1.append(p1+1)
            player2.append(p2+1)
            
            outcome.append(*random.choices([1,0],weights=[win_rate,1-win_rate]))

        return player1,player2,outcome
    
    elif style == 'pygm':
        games=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )
            
            games.append((p1,p2,*random.choices([1,-1],weights=[win_rate,1-win_rate])))
        return games
    
    assert False 


In [0]:
def skill_MSE(predicted_skill, true_skill):
    return sum( (predicted_skill[i]-true_skill[i])**2 for i in range(len(predicted_skill)) )/len(predicted_skill)

def prediction_accuracy(bel, X, Pwin, true_players, num_valid_game=1000, scale=0.3):
    
    valid_games = generate_games(true_players, num_valid_game, scale, style='pygm')

    acc=0

    for g in valid_games:

        i,j,result=int(g[0]),int(g[1]),int(g[2])
        if i<j:
            prediction = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() 
        else:
            prediction = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum()

        acc+=int( (prediction >= 0.5 and result==1) or (prediction<0.5 and result==-1) )

    return acc/num_valid_game

In [0]:

scale=0.5
num_of_players=10

nplayers = num_of_players
nlevels = 10   # let's say 10 discrete skill levels

result_skill_MSE_10={}
result_prediction_10={}

for num_of_games in [10,20,50,100,500,700,1000,3000,5000]:
    
    trials_result_skill=[]
    trials_result_prediction=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,nlevels)
        train_games = generate_games(true_players, num_of_games, scale, style='pygm')

        print("true skills:",true_players)

        # nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players

        # Make variables for each player; value = skill level
        X = [None]*nplayers
        for i in range(nplayers):
            X[i] = gm.Var(i, nlevels)   

        # Information from each game: what does Pi winning over Pj tell us?
        #    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
        Pwin = np.zeros( (nlevels,nlevels) )
        for i in range(nlevels):
            for j in range(nlevels):
                diff = i-j                   # find the advantage of Pi over Pj, then 
                Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

        # before any games, uniform belief over skill levels for each player:
        factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

        # Now add the information from each game:
        for g in train_games:
            P1,P2,win = g[0],g[1],g[2]
            if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
            factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )


        starting=time.time()
        model = gm.GraphModel(factors)
        model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)
        print("spent:",round(time.time()-starting,3))


        if model.nvar < 0:       # for very small models, we can do brute force inference:
            jt = model.joint()
            jt /= jt.sum()       # normalize the distribution and marginalize the table
            bel = [jt.marginal([i]) for i in range(nplayers)] 

        else:                    # otherwise we need to use some approximate inference:
            from pyGM.messagepass import LBP, NMF
            #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
            lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field


        predicted_players = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]

        trials_result_skill.append( skill_MSE(predicted_players,true_players) )
        trials_result_prediction.append( prediction_accuracy(bel, X, Pwin, true_players, num_valid_game=1000, scale=scale) )

    result_skill_MSE_10[num_of_games]=trials_result_skill
    result_prediction_10[num_of_games]=trials_result_prediction
    


starting num_of_games = 10
true skills: [6.1, 1.21, 1.26, 0.34, 7.43, 2.3, 8.62, 4.7, 1.17, 0.5]
spent: 0.005
Iter 0: -10.87707604119268
Iter 1: -6.4078985293287705
Iter 2: -6.207061388441227
Iter 3: -6.205097567660567
Iter 4: -6.205076529597306
Iter 5: -6.205076202277907
Iter 6: -6.2050761925865725
Iter 7: -6.205076192170646
Iter 8: -6.2050761921511
Iter 9: -6.205076192150172
Iter 10: -6.205076192150119
true skills: [0.34, 2.14, 3.74, 6.34, 3.83, 3.81, 8.76, 6.67, 7.62, 9.35]
spent: 0.005
Iter 0: -10.877076041192684
Iter 1: -7.248773933948197
Iter 2: -7.20511863701533
Iter 3: -7.204910329592925
Iter 4: -7.204880847466752
Iter 5: -7.204879245880836
Iter 6: -7.204879152029858
Iter 7: -7.204879146283231
Iter 8: -7.204879145884055
Iter 9: -7.204879145855885
Iter 10: -7.20487914585391
true skills: [2.93, 2.98, 5.89, 6.84, 3.08, 2.62, 4.45, 8.56, 3.16, 4.41]
spent: 0.005
Iter 0: -10.87707604119268
Iter 1: -8.21592556554905
Iter 2: -8.181071024368372
Iter 3: -8.177757185719168
Iter 4: -8.177

10 players result

In [0]:
print(result_skill_MSE_10)
print(result_prediction_10)

{10: [8.13335597471637, 8.682231084377218, 3.567016040101362, 4.20730041954664, 3.9909155276753028, 1.5530444517436228, 8.802384547417583, 3.407844775805853, 5.2087311920969395, 8.345158570635133], 20: [4.237353030624334, 5.110289908586866, 3.505512605550288, 2.949317097550754, 5.314996855577973, 3.6776414491402227, 7.214536171882438, 5.6796228969241795, 1.441079401080878, 1.7956035478251877], 50: [3.1471366199045425, 2.0811211848441102, 0.8771372722647751, 3.2505737754014006, 2.000905523998185, 2.7830440790098976, 2.3318701186313566, 2.3371275391795523, 1.3148143020655283, 4.1126595002371955], 100: [1.8891897203936527, 0.6565226851324699, 1.2561593102215085, 3.2432873766681696, 1.2546167113709357, 0.2998792798275221, 1.6291018433901105, 1.1210802556163921, 0.8660547679830914, 0.8347579146549178], 500: [0.7710687861581234, 0.9298821421225231, 9.185890193832481, 3.3466151462102025, 0.4229155884287897, 1.3929049275204994, 0.38644537692400577, 2.523567208011317, 0.60565599537596, 0.435849

In [0]:

scale=0.5
num_of_players=20

nplayers = num_of_players
nlevels = 10   # let's say 10 discrete skill levels

result_skill_MSE_20={}
result_prediction_20={}

for num_of_games in [10,20,50,100,500,700,1000,3000,5000]:
    
    trials_result_skill=[]
    trials_result_prediction=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,nlevels)
        train_games = generate_games(true_players, num_of_games, scale, style='pygm')

        print("true skills:",true_players)

        # nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players

        # Make variables for each player; value = skill level
        X = [None]*nplayers
        for i in range(nplayers):
            X[i] = gm.Var(i, nlevels)   

        # Information from each game: what does Pi winning over Pj tell us?
        #    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
        Pwin = np.zeros( (nlevels,nlevels) )
        for i in range(nlevels):
            for j in range(nlevels):
                diff = i-j                   # find the advantage of Pi over Pj, then 
                Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

        # before any games, uniform belief over skill levels for each player:
        factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

        # Now add the information from each game:
        for g in train_games:
            P1,P2,win = g[0],g[1],g[2]
            if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
            factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )


        starting=time.time()
        model = gm.GraphModel(factors)
        model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)
        print("spent:",round(time.time()-starting,3))


        if model.nvar < 0:       # for very small models, we can do brute force inference:
            jt = model.joint()
            jt /= jt.sum()       # normalize the distribution and marginalize the table
            bel = [jt.marginal([i]) for i in range(nplayers)] 

        else:                    # otherwise we need to use some approximate inference:
            from pyGM.messagepass import LBP, NMF
            #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
            lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field


        predicted_players = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]

        trials_result_skill.append( skill_MSE(predicted_players,true_players) )
        trials_result_prediction.append( prediction_accuracy(bel, X, Pwin, true_players, num_valid_game=1000, scale=scale) )

    result_skill_MSE_20[num_of_games]=trials_result_skill
    result_prediction_20[num_of_games]=trials_result_prediction
    


starting num_of_games = 10
true skills: [7.67, 7.59, 8.62, 0.06, 8.4, 9.56, 3.13, 7.86, 5.84, 5.18, 3.08, 0.62, 8.85, 3.32, 5.91, 4.04, 9.69, 4.22, 3.66, 4.06]
spent: 0.006
Iter 0: -10.87707604119268
Iter 1: -6.008775514081084
Iter 2: -5.880610936402864
Iter 3: -5.87966578786531
Iter 4: -5.879648118700209
Iter 5: -5.879647761834055
Iter 6: -5.879647754521882
Iter 7: -5.87964775437188
Iter 8: -5.879647754368798
Iter 9: -5.87964775436873
Iter 10: -5.879647754368722
true skills: [6.08, 0.89, 4.08, 9.04, 5.28, 7.2, 9.07, 8.31, 0.39, 7.26, 8.41, 7.76, 4.28, 0.18, 5.09, 6.77, 4.7, 4.19, 9.14, 4.91]
spent: 0.006
Iter 0: -10.87707604119268
Iter 1: -7.082968382648535
Iter 2: -7.028600015389486
Iter 3: -7.028446158421595
Iter 4: -7.028445022602789
Iter 5: -7.0284450117915735
Iter 6: -7.028445011660919
Iter 7: -7.028445011659031
Iter 8: -7.028445011659006
Iter 9: -7.028445011659009
Iter 10: -7.028445011659006
true skills: [10.0, 9.16, 4.51, 9.29, 4.12, 0.28, 3.14, 4.56, 6.07, 0.56, 0.89, 7.03, 6.

20 players result

In [0]:
print(result_skill_MSE_20)
print(result_prediction_20)

{10: [6.045697127886505, 7.78486782338221, 8.452100082535864, 8.717851890507578, 10.948017502263543, 11.146373989959212, 6.204621341990442, 5.701267200307193, 8.111648467545796, 6.883997006753288], 20: [7.795475279028632, 7.3377336705966885, 8.52263538425918, 4.11754394117589, 6.297883918881931, 5.7900188942305295, 6.040025140446856, 6.8847663544919815, 6.199099705235869, 8.377339726757752], 50: [3.857647660410636, 6.063663179434068, 3.027988444130567, 3.0122140146149787, 4.432952163820723, 2.3896115294647684, 2.8247288011897442, 5.639139285611661, 3.7788764855517605, 4.29524475165667], 100: [4.2654164639377, 1.9302580298688679, 1.8930075493348812, 4.038569221167242, 2.1201056410714543, 1.7562164451678572, 2.3846515519286045, 1.8676021534938798, 3.8053192866955654, 2.566850545520846], 500: [0.49444311818476105, 1.749817533218972, 0.3736972430566641, 1.132408886412356, 0.661987843611135, 0.48643562933364376, 0.903626302375655, 0.5053798260425656, 1.8612650506663417, 0.47617520647689837]

In [0]:

scale=0.5
num_of_players=30

nplayers = num_of_players
nlevels = 10   # let's say 10 discrete skill levels

result_skill_MSE_30={}
result_prediction_30={}

for num_of_games in [10,20,50,100,500,700,1000,3000,5000]:
    
    trials_result_skill=[]
    trials_result_prediction=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,nlevels)
        train_games = generate_games(true_players, num_of_games, scale, style='pygm')

        print("true skills:",true_players)

        # nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players

        # Make variables for each player; value = skill level
        X = [None]*nplayers
        for i in range(nplayers):
            X[i] = gm.Var(i, nlevels)   

        # Information from each game: what does Pi winning over Pj tell us?
        #    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
        Pwin = np.zeros( (nlevels,nlevels) )
        for i in range(nlevels):
            for j in range(nlevels):
                diff = i-j                   # find the advantage of Pi over Pj, then 
                Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

        # before any games, uniform belief over skill levels for each player:
        factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

        # Now add the information from each game:
        for g in train_games:
            P1,P2,win = g[0],g[1],g[2]
            if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
            factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )


        starting=time.time()
        model = gm.GraphModel(factors)
        model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)
        print("spent:",round(time.time()-starting,3))


        if model.nvar < 0:       # for very small models, we can do brute force inference:
            jt = model.joint()
            jt /= jt.sum()       # normalize the distribution and marginalize the table
            bel = [jt.marginal([i]) for i in range(nplayers)] 

        else:                    # otherwise we need to use some approximate inference:
            from pyGM.messagepass import LBP, NMF
            #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
            lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field


        predicted_players = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]

        trials_result_skill.append( skill_MSE(predicted_players,true_players) )
        trials_result_prediction.append( prediction_accuracy(bel, X, Pwin, true_players, num_valid_game=1000, scale=scale) )

    result_skill_MSE_30[num_of_games]=trials_result_skill
    result_prediction_30[num_of_games]=trials_result_prediction
    


starting num_of_games = 10
true skills: [5.33, 2.94, 3.05, 3.88, 0.17, 7.89, 0.46, 6.53, 2.69, 2.5, 3.32, 0.28, 2.34, 7.21, 9.41, 9.35, 8.35, 8.6, 6.28, 3.12, 8.43, 3.6, 8.58, 4.09, 9.03, 5.7, 8.04, 9.26, 3.96, 5.27]
spent: 0.008
Iter 0: -10.877076041192673
Iter 1: -7.842972514650132
Iter 2: -7.768666863451008
Iter 3: -7.768312977733741
Iter 4: -7.768299404086594
Iter 5: -7.768298968701903
Iter 6: -7.768298954845096
Iter 7: -7.768298954401017
Iter 8: -7.768298954386747
Iter 9: -7.768298954386276
Iter 10: -7.768298954386253
true skills: [5.71, 6.56, 7.76, 1.01, 2.27, 7.85, 7.75, 8.23, 4.98, 6.04, 0.09, 9.16, 7.47, 8.58, 6.3, 7.85, 2.28, 4.77, 2.7, 2.85, 10.0, 9.41, 5.78, 5.35, 7.59, 2.25, 3.98, 9.02, 7.32, 7.67]
spent: 0.008
Iter 0: -10.877076041192673
Iter 1: -6.89844134720148
Iter 2: -6.794593297469754
Iter 3: -6.790869956929049
Iter 4: -6.790809985817999
Iter 5: -6.790808896159251
Iter 6: -6.790808875063613
Iter 7: -6.790808874641605
Iter 8: -6.790808874633018
Iter 9: -6.790808874632

30 players result

In [0]:
print(result_skill_MSE_30)
print(result_prediction_30)

{10: [8.834097132663713, 8.666208733765389, 7.321370295308273, 10.141844478637784, 8.266000523032625, 7.727498253692624, 10.187804465333677, 5.891728568730087, 7.044213940362623, 6.157119620607801], 20: [8.692928491429806, 9.068993101445674, 6.303501252729824, 5.8039828069779436, 6.776274495597191, 4.72356797755283, 6.563381880450036, 6.593273069788686, 7.605100596654089, 8.400899334035522], 50: [6.330816746731398, 4.897741924913485, 4.72516984090611, 5.375458612654307, 4.301580166245022, 3.351444955550703, 7.744979481278813, 4.8508020842063795, 5.133598363435029, 5.897450034811817], 100: [3.4438812634774156, 2.370927825469347, 4.72584232692502, 2.6172591697642718, 4.211313318699154, 2.7148856013441565, 5.191349298283579, 2.8603023750788763, 4.434219186590435, 2.7597240218807944], 500: [0.32429664100746014, 1.0459837766053384, 0.6119132064461181, 0.9160944640060784, 1.4247532147084727, 1.7922837184257823, 0.5713393788247666, 1.2046644519613487, 0.9273831109533907, 0.6108657082514095], 

In [0]:
scale=0.5
num_of_players=40

nplayers = num_of_players
nlevels = 10   # let's say 10 discrete skill levels

result_skill_MSE_40={}
result_prediction_40={}

for num_of_games in [10,20,50,100,500,700,1000,3000,5000]:
    
    trials_result_skill=[]
    trials_result_prediction=[]
    print("starting num_of_games =",num_of_games)
    for trials in range(10):
        true_players = generate_players(num_of_players,nlevels)
        train_games = generate_games(true_players, num_of_games, scale, style='pygm')

        print("true skills:",true_players)

        # nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players

        # Make variables for each player; value = skill level
        X = [None]*nplayers
        for i in range(nplayers):
            X[i] = gm.Var(i, nlevels)   

        # Information from each game: what does Pi winning over Pj tell us?
        #    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
        Pwin = np.zeros( (nlevels,nlevels) )
        for i in range(nlevels):
            for j in range(nlevels):
                diff = i-j                   # find the advantage of Pi over Pj, then 
                Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

        # before any games, uniform belief over skill levels for each player:
        factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

        # Now add the information from each game:
        for g in train_games:
            P1,P2,win = g[0],g[1],g[2]
            if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
            factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )


        starting=time.time()
        model = gm.GraphModel(factors)
        model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)
        print("spent:",round(time.time()-starting,3))


        if model.nvar < 0:       # for very small models, we can do brute force inference:
            jt = model.joint()
            jt /= jt.sum()       # normalize the distribution and marginalize the table
            bel = [jt.marginal([i]) for i in range(nplayers)] 

        else:                    # otherwise we need to use some approximate inference:
            from pyGM.messagepass import LBP, NMF
            #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
            lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field


        predicted_players = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]

        trials_result_skill.append( skill_MSE(predicted_players,true_players) )
        trials_result_prediction.append( prediction_accuracy(bel, X, Pwin, true_players, num_valid_game=1000, scale=scale) )

    result_skill_MSE_40[num_of_games]=trials_result_skill
    result_prediction_40[num_of_games]=trials_result_prediction
    


starting num_of_games = 10
true skills: [4.66, 4.74, 7.15, 4.52, 4.16, 4.98, 1.88, 7.77, 9.3, 3.66, 7.64, 8.67, 2.17, 4.72, 3.54, 5.86, 1.92, 7.69, 6.08, 4.71, 8.78, 7.67, 1.26, 5.27, 6.33, 1.54, 7.76, 0.94, 9.71, 5.46, 7.85, 8.04, 6.46, 3.95, 0.87, 4.03, 3.51, 7.57, 9.6, 3.34]
spent: 0.014
Iter 0: -10.877076041192687
Iter 1: -7.1922978545582765
Iter 2: -7.056217575133424
Iter 3: -7.053188209807541
Iter 4: -7.053124912683298
Iter 5: -7.053123779536314
Iter 6: -7.053123759790913
Iter 7: -7.0531237594481375
Iter 8: -7.05312375944219
Iter 9: -7.05312375944208
Iter 10: -7.05312375944209
true skills: [8.76, 2.08, 2.56, 0.46, 8.1, 0.66, 2.69, 6.9, 7.11, 9.29, 9.16, 3.05, 8.66, 3.66, 8.7, 4.64, 9.29, 2.81, 9.99, 6.02, 2.71, 9.6, 2.46, 1.65, 7.95, 2.51, 9.45, 3.82, 0.04, 6.27, 3.44, 7.8, 2.59, 1.49, 5.46, 5.1, 9.71, 8.17, 5.02, 4.36]
spent: 0.009
Iter 0: -10.877076041192682
Iter 1: -6.7643748136203135
Iter 2: -6.66903615712409
Iter 3: -6.668582866804138
Iter 4: -6.668579606425406
Iter 5: -6.66

40 players result

In [0]:
print(result_skill_MSE_40)
print(result_prediction_40)

{10: [6.583778046137361, 8.772709093927372, 6.054275909722918, 6.161750834713718, 8.011923660801653, 10.647616112913516, 7.214489983693724, 6.886263581213301, 8.957160214798288, 8.651429089904921], 20: [5.123765653042564, 7.694160797873103, 5.776628614024816, 7.234150365410352, 5.059942015787388, 7.222543365749023, 6.913556416076654, 9.09840621303422, 5.350156304944808, 6.523553086844508], 50: [4.34053423350553, 4.734667052974589, 5.793142295062335, 6.272005888374801, 5.868727052561829, 4.957280549610784, 4.620182118568183, 6.342391935609186, 6.637958575688016, 7.162635824101082], 100: [4.729577316770932, 4.67992748317577, 4.906806733567703, 5.655049112916198, 4.574308513337964, 2.8181216657393047, 3.3395327318634345, 3.781626887429403, 3.025596669666664, 3.917267619830767], 500: [0.8915720111368168, 1.042229611682951, 1.2137715624261158, 0.8571727142519316, 1.1798391949510234, 0.8688086298575749, 1.0626557310880123, 1.2030636993747983, 0.914956646497119, 0.8662315588519904], 700: [0.9

In [6]:
result_skill_MSE_50_100=[]
result_prediction_50_100=[]
for num_of_players in [50,60,70,80,90,100,150,200,300,500]:
  scale=0.5


  nplayers = num_of_players
  nlevels = 10   # let's say 10 discrete skill levels

  result_skill_MSE={}
  result_prediction={}

  for num_of_games in [10,20,50,100,500,700,1000,3000,5000]:
      
      trials_result_skill=[]
      trials_result_prediction=[]
      print("starting num_of_games =",num_of_games)
      for trials in range(10):
          true_players = generate_players(num_of_players,nlevels)
          train_games = generate_games(true_players, num_of_games, scale, style='pygm')

          print("true skills:",true_players)

          # nplayers = max( [max(g[0],g[1]) for g in games] )+1 #number of players

          # Make variables for each player; value = skill level
          X = [None]*nplayers
          for i in range(nplayers):
              X[i] = gm.Var(i, nlevels)   

          # Information from each game: what does Pi winning over Pj tell us?
          #    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
          Pwin = np.zeros( (nlevels,nlevels) )
          for i in range(nlevels):
              for j in range(nlevels):
                  diff = i-j                   # find the advantage of Pi over Pj, then 
                  Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

          # before any games, uniform belief over skill levels for each player:
          factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

          # Now add the information from each game:
          for g in train_games:
              P1,P2,win = g[0],g[1],g[2]
              if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
              factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )


          starting=time.time()
          model = gm.GraphModel(factors)
          model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)
          print("spent:",round(time.time()-starting,3))


          if model.nvar < 0:       # for very small models, we can do brute force inference:
              jt = model.joint()
              jt /= jt.sum()       # normalize the distribution and marginalize the table
              bel = [jt.marginal([i]) for i in range(nplayers)] 

          else:                    # otherwise we need to use some approximate inference:
              from pyGM.messagepass import LBP, NMF
              #lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
              lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field


          predicted_players = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]

          trials_result_skill.append( skill_MSE(predicted_players,true_players) )
          trials_result_prediction.append( prediction_accuracy(bel, X, Pwin, true_players, num_valid_game=1000, scale=scale) )

      result_skill_MSE[num_of_games]=trials_result_skill
      result_prediction[num_of_games]=trials_result_prediction
  result_skill_MSE_50_100.append(result_skill_MSE)
  result_prediction_50_100.append(result_prediction)

Streaming output truncated to the last 5000 lines.
Iter 3: -566.0538670612303
Iter 4: -566.0080035678839
Iter 5: -565.9825424261727
Iter 6: -565.9684883883442
Iter 7: -565.960765044643
Iter 8: -565.9565234886544
Iter 9: -565.9541940449615
Iter 10: -565.9529146459685
true skills: [3.95, 6.71, 3.16, 9.87, 6.27, 3.9, 1.1, 9.75, 6.98, 0.97, 5.66, 2.16, 6.78, 1.18, 3.86, 0.82, 9.2, 0.57, 2.09, 2.41, 0.08, 9.05, 3.45, 9.07, 0.56, 9.54, 1.62, 8.54, 3.86, 1.77, 1.57, 5.99, 4.33, 5.06, 9.17, 4.76, 5.24, 9.75, 7.52, 9.14, 1.96, 7.38, 4.13, 0.35, 3.12, 2.68, 3.19, 4.82, 4.79, 4.23, 4.09, 8.83, 0.79, 8.2, 5.11, 0.88, 2.2, 1.15, 1.55, 4.72, 6.99, 0.03, 3.46, 4.47, 0.58, 0.67, 6.01, 0.51, 9.33, 8.11, 9.84, 8.0, 6.35, 6.99, 9.21, 8.51, 3.52, 8.92, 1.86, 1.73, 1.9, 5.45, 3.47, 0.19, 9.51, 2.16, 7.2, 6.46, 4.44, 9.26, 5.53, 4.09, 0.65, 0.2, 2.82, 0.98, 1.14, 3.94, 9.89, 6.66]
spent: 0.532
Iter 0: -1087.7076041192631
Iter 1: -533.9709811537948
Iter 2: -526.2636459850613
Iter 3: -525.8741922160336
Iter 4

[50,60,70,80,90,100,150,200,300,500] players result
it's  10 dictionarys inside a list

In [7]:
print(result_skill_MSE_50_100)
print(result_prediction_50_100)

[{10: [9.45303647512616, 8.130556780781061, 8.137767229497026, 8.959792678318067, 8.770557681728558, 6.515678451358173, 9.629552811821974, 9.123709083922797, 9.918936186380227, 8.472116065314884], 20: [7.815111822208823, 8.732999961739644, 5.114951362041567, 8.754552739667952, 6.290210408431094, 7.145228901260119, 7.695493492984643, 7.385889810821078, 7.081469739482449, 7.8926061999335], 50: [7.0241088375517755, 5.143130984535994, 4.323455416759553, 4.212658592221732, 4.210170129073427, 6.902079113217802, 4.251999121707531, 4.674210965258609, 6.287584467075826, 6.707326558223922], 100: [6.551833481681302, 7.128162791495658, 2.935155021569126, 4.5234045833899, 4.066708768853796, 4.4678944655315505, 3.796951673816453, 4.139088997478238, 4.5011026667824465, 3.58303624691564], 500: [1.1665984824907536, 1.9732296467799018, 1.0156207719204813, 1.468346311258417, 1.301623150279457, 1.6057857644910445, 0.653514975179109, 1.337691899824502, 1.3713455483862733, 2.0242633566007204], 700: [0.64938